In [130]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
from pprint import pprint
from lxml import html
from time import sleep

In [135]:
PREPARED_DATASET_PATH = './vacansy.csv'

In [121]:
url = 'https://hh.ru/vacancies/data-analyst'

In [3]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',}

In [119]:
def razbor_str (stroka_in):
    
    if len(stroka_in) == 0:
        return [None,None,None]
    stroka_in = stroka_in.replace(' ', '')
    stroka_in = stroka_in.replace('\u202f', '')
    if 'руб.' in stroka_in:
        stroka_out = [None,None,'RU']
        stroka_in = stroka_in.replace('руб.', '')
    elif 'USD' in stroka_in:
        stroka_out = [None,None,'USD']
        stroka_in = stroka_in.replace('USD', '')
    elif 'EUR' in stroka_in:
        stroka_out = [None,None,'EUR']
        stroka_in = stroka_in.replace('EUR', '')    
    if 'от' in stroka_in:        
        stroka_out[0] = int(stroka_in[2:])
        stroka_out[1] = None
        return stroka_out
    elif 'до' in stroka_in:
        stroka_out[0] = None
        stroka_out[1] = int(stroka_in[2:])
        return stroka_out
        
    stroka_kort = stroka_in.split('–')      
    stroka_out[0] = int(stroka_kort[0])
    stroka_out[1] = int(stroka_kort[1])
    return stroka_out

In [128]:
params = {'page': 0}
pages = 3
n = 0
lstDict_vacancy =[]
Keys = ['link','title','min','max','val','source']
while params['page'] <= pages:
    sleep(3)
    response = requests.get(url=url, headers=headers, params = params )
    dom = BS(response.text, 'html.parser')

    for link in dom.find_all('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}):
        l_in = []
        l_in.append(link['href'])
        l_in.append(link.text)
        salary_str = razbor_str(link.parent.parent.parent.next_sibling.next_sibling.text)
        
        for i, v in enumerate(salary_str):
            l_in.append(salary_str[i])
        l_in.append('hh.ru')    
        Dict_vacancy = dict(zip(Keys,l_in))
        lstDict_vacancy.append(Dict_vacancy)

    params['page'] += 1
    

[{'link': 'https://spb.hh.ru/vacancy/69487131?query=data%20analyst', 'title': 'Data Analyst', 'min': 100000, 'max': 150000, 'val': 'RU', 'source': 'hh.ru'}, {'link': 'https://spb.hh.ru/vacancy/54721740?query=data%20analyst', 'title': 'Младший аналитик данных (Junior data analyst)', 'min': 100000, 'max': None, 'val': 'RU', 'source': 'hh.ru'}, {'link': 'https://spb.hh.ru/vacancy/69175957?query=data%20analyst', 'title': 'Data Scientist', 'min': 200000, 'max': None, 'val': 'RU', 'source': 'hh.ru'}, {'link': 'https://spb.hh.ru/vacancy/69519930?query=data%20analyst', 'title': 'Product Data Analyst', 'min': None, 'max': None, 'val': None, 'source': 'hh.ru'}, {'link': 'https://spb.hh.ru/vacancy/69076712?query=data%20analyst', 'title': 'Аналитик BI', 'min': 100000, 'max': None, 'val': 'RU', 'source': 'hh.ru'}, {'link': 'https://spb.hh.ru/vacancy/69492359?query=data%20analyst', 'title': 'Middle/Senior Data Scientist', 'min': 200000, 'max': 400000, 'val': 'RU', 'source': 'hh.ru'}, {'link': 'https

In [131]:
df = pd.DataFrame(lstDict_vacancy)

In [133]:
df.head()

,link,title,min,max,val,source
0,https://spb.hh.ru/vacancy/69487131?query=data%...,Data Analyst,100000.0,150000.0,RU,hh.ru
1,https://spb.hh.ru/vacancy/54721740?query=data%...,Младший аналитик данных (Junior data analyst),100000.0,NaN,RU,hh.ru
2,https://spb.hh.ru/vacancy/69175957?query=data%...,Data Scientist,200000.0,NaN,RU,hh.ru
3,https://spb.hh.ru/vacancy/69519930?query=data%...,Product Data Analyst,NaN,NaN,None,hh.ru
4,https://spb.hh.ru/vacancy/69076712?query=data%...,Аналитик BI,100000.0,NaN,RU,hh.ru


In [136]:
df.to_csv(PREPARED_DATASET_PATH, index=False, encoding='cp1251', sep=';')